# Origin Logger
> A closer look at this Agent Tesla successor 

- toc: true 
- badges: true
- categories: [dotnet,origin logger,agenttesla,config]

## Overview

This is a .NET stealer that is possibly a clone or new version of AgentTesla. It has been sold on public grayware sites such as `fudsender[.]com`.

![](https://i.imgur.com/D3YOfYs.png)

There is even a YouTube commercial for the stealer!

[https://www.youtube.com/watch?v=o-MDujYrtto](https://www.youtube.com/watch?v=o-MDujYrtto)


### Sample

`b1114c27beb856eae1f9fba0a880450702b7bda007f0fbacc4d5df561d83ec88` [UnpacMe](https://www.unpac.me/results/7038ff4e-4114-479d-84b1-b2dfa4babcaf?hash=b1114c27beb856eae1f9fba0a880450702b7bda007f0fbacc4d5df561d83ec88#/)

### References 

- [OriginBotnet Spreads via Malicious Word Document](https://www.fortinet.com/blog/threat-research/originbotnet-spreads-via-malicious-word-document)
- [OriginLogger: A Look at Agent Tesla’s Successor](https://unit42.paloaltonetworks.com/originlogger/)
- [How To Develop Yara Rules for .NET Malware Using IL ByteCodes](https://embee-research.ghost.io/yara-rule-development-il-instructions-in-redline-malware/)


## Analysis

- The sample we are analyzing does not have encrypted strings and the C2 is in plaintext `https[:]//jwgruop[.]com/login/`
- The HTTP C2 uses a hard coded header value `x-key` which can be used combined with the other agent tesla features to identify the sample
- 





### Settings

```c
        public static bool Keylogger = true;
        public static int LoggerInterval = 10;
        public static bool PasswordRecovery = true;
        public static bool CookieRecovery = true;
        public static int TenantId = 4319;
        public static string PcHwid = "";
        public static string PcName = "";
        public static string AsmLocation = "";
        public static int CheckInterval = 5000;
        public static string PublicKey = "<RSAKeyValue><Modulus>rAoFBxYpptcD05ezf2if4RlryyqIBxjA3797MhTklk3xemvM9Y+Wle6P5hBHzSu95KQmgea8lzbH7RA3HOb1UvKErFeJ3jT5XhsqaT4AcDLRQraXpmxz+rzW0GK+ZvJFYmIL+18m4j/TAHjT40L0wEcfu2qzrIzroEFupW/oELM=</Modulus><Exponent>AQAB</Exponent></RSAKeyValue>";
        public static string PublicUserAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0";
        public static bool AppAddStartup = Convert.ToBoolean("false");
        public static bool HideFileStartup = Convert.ToBoolean("false");
        public static string AppStartupFullPath = "";
        public static string StartupDirectoryPath = "";
        public static string StartupEnvName = "appdata";
        public static string StartupDirectoryName = "hhlquS";
        public static string StartupInstallationName = "hhlquS.exe";
        public static string StartupRegName = "hhlquS";
        public static string DownloadFolderName = "4yyaqgto.m0g";
        public static string DownloadDirectoryPath = "";
        public static string CurrentUrl = "";
        public static string[] FallbackUrls = new string[] { "https://nitrosoftwares.shop/gate" };
```




### Yara Rule


```
rule OriginLogger {
    strings:

        $s1 = "select * from  Win32_Processor" wide 
        $s2 = "FileArgs" wide 
        $s3 = "StringFileInfo" wide 
        $s4 = "select * from  Win32_VideoController" wide 
        $s5 = "Win32_NetworkAdapterConfiguration" wide 
        $s6 = "downloadexecute" wide 
        $x = "x-key" wide nocase 
        $dl_bytes = {7E ?? ?? ?? ?? 72 ?? ?? ?? ?? 03 04 28 ?? ?? ?? ?? 0A 02 06 28 ?? ?? ?? ?? 06 28 }

    condition:
        filesize < 50KB and
        (
            (
                5 of ($s*) and
                $x
            ) or
            (
                $dl_bytes
            )
        ) 
        
}
```






## C2 Extraction

For now we only have samples that do not have encrypted strings so c2 extraction is just a simple regex.


In [17]:
import re

file_data = open('/tmp/origin.bin', 'rb').read()


def filter_printable(data):
    out = []
    for c in data:
        if c < 33 or c > 126 or c == 92:
            continue
        out.append(c)
    return bytes(out)




def get_c2s(file_data):
    c2s = []
    for s in file_data.split(b'h\x00t\x00t\x00'):
        if s[:2] == b'p\x00':
            tmp = s.split(b'\x00\x00')[0]
            out = b'htt' + filter_printable(tmp.replace(b'\x00',b''))
            if len(out) < 4 or len(out) > 50:
                continue
            #print(out)
            c2s.append(out.decode('utf-8'))
    return c2s


get_c2s(file_data)

['http://jwgruop.com/gate']

In [18]:

# import required module
import os
# assign directory
directory = '/tmp/samples'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        file_data = open(f, 'rb').read()
        print(get_c2s(file_data))

/tmp/samples/74c7b909f9fd299853c459b903f7e60c6c2e2517550b26b87f270404895e1284
['https://mmelak.com/gate']
/tmp/samples/ee8a244c904756bdc3987fefc844596774437bcc50d4022ddcc94e957cab6a11
['https://nitrosoftwares.shop/gate']
/tmp/samples/18a9e61a082feb6c7e04d9aef4a0a0d5db926158562e7d3d408413fc4efdbbac
['https://grdhfour.shop/gate']
/tmp/samples/9eb6b58e9cdaca568b1d5dfe7912d454fda7e14716ac68b641794612f6d40044
['http://ltm-canada.com/gate']
/tmp/samples/83542943a06c095719a158597c0c11b76829ef8375e21cc5d2ea0b17de6713a1
['https://prefiex.com.co/gate']
/tmp/samples/4ad9f85d1b2acc62e8be5fa82d1c7c0e2321c0c1cba30253fd5d5b0af864c076
['https://veit-intl.com/gate']
/tmp/samples/647c1751e9dbd59f7b75a0ec72e21e128450422f7f9aa0f296344a1c9e9fff21
['http://yingxnimetals.com/gate']
/tmp/samples/2a2542a79cf50b7f28305912b00f5acff3744eac20b5fdbf51fc18eadbb4bfd4
['https://nitrosoftwares.shop/gate']
/tmp/samples/3a7ba1b580177fe12f5560bab6af386ff1c7ed6dd56b190f8e574475bb35d970
['https://nitrosoftwares.shop/gate']
